<a href="https://colab.research.google.com/github/ZahraFayyaz/3dshape-vqvae-pyTorch/blob/main/train_vqvae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import modules

In [1]:
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.1 MB/s eta 0:00:00


In [2]:
import sys, os, yaml
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

# Enter the foldername where all modules are stored together
FOLDERNAME = 'INI - Generative Episodic Memory/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Change the working/current directory
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/My Drive/INI - Generative Episodic Memory


In [3]:

import torch
import torch.nn as nn
from data_loader import load_data
from vqvae import get_model
# import argparse
import random
# import shutil
import cv2
import torchvision
import numpy as np
from tqdm import tqdm
from torch.optim import Adam
from torchvision.utils import make_grid
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_for_one_epoch(epoch_idx, model, data_loader, optimizer, crtierion, config, save_indices=False):
    r"""
    Method to run the training for one epoch.
    :param epoch_idx: iteration number of current epoch
    :param model: VQVAE model
    :param data_loader: Data loder for mnist
    :param optimizer: optimzier to be used taken from config
    :param crtierion: For computing the loss
    :param config: configuration for the current run
    :return:
    """
    recon_losses = []
    codebook_losses = []
    commitment_losses = []
    losses = []
    # List to collect indices of all images
    all_indices = []

    count = 0
    for im, label in tqdm(data_loader, desc='Training', leave=False): # Ignore the label in DataLoader
        im = im.float().to(device)
        optimizer.zero_grad()
        model_output = model(im)
        output = model_output['generated_image']
        quantize_losses = model_output['quantized_losses']
        z_q = model_output['quantized_output']
        indices = model_output['quantized_indices']

        # if config['train_params']['save_training_image']:
        #     cv2.imwrite('input.jpeg', (255 * (im.detach() + 1) / 2).cpu().permute(0, 1, 2, 3).numpy().astype(np.uint8)) #(255 * (im.detach() + 1) / 2).cpu().permute((0, 2, 3, 1)).numpy()[0]
        #     cv2.imwrite('output.jpeg', (255 * (output.detach() + 1) / 2).cpu().permute(0, 1, 2, 3).numpy().astype(np.uint8)) #(255 * (output.detach() + 1) / 2).cpu().permute((0, 2, 3, 1)).numpy()[0]

        recon_loss = crtierion(output, im)
        loss = (config['train_params']['reconstruction_loss_weight']*recon_loss +
                config['train_params']['codebook_loss_weight']*quantize_losses['codebook_loss'] +
                config['train_params']['commitment_loss_weight']*quantize_losses['commitment_loss'])
        recon_losses.append(recon_loss.item())
        codebook_losses.append(config['train_params']['codebook_loss_weight']*quantize_losses['codebook_loss'].item())
        commitment_losses.append(quantize_losses['commitment_loss'].item())
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

        # Collect indices for all images
        all_indices.append(indices.cpu())

    # Save indices if save_indices is True
    if save_indices:
        all_indices_tensor = torch.cat(all_indices, dim=0)

    print('Finished epoch: {} | Recon Loss : {:.4f} | Codebook Loss : {:.4f} | Commitment Loss : {:.4f}'.
          format(epoch_idx + 1,
                 np.mean(recon_losses),
                 np.mean(codebook_losses),
                 np.mean(commitment_losses)))
    return np.mean(losses)#, all_indices_tensor


def train(config_path, sample=None):
    ######## Read the config file #######
    with open(config_path, 'r') as file:
        try:
            config = yaml.safe_load(file)
        except yaml.YAMLError as exc:
            print(exc)
    # print(config)
    #######################################

    ######## Set the desired seed value #######
    seed = config['train_params']['seed']
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if device == 'cuda':
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        # print(args.seed)
    #######################################

    # Create the model and dataset
    model = get_model(config).to(device)
    data_loader = load_data(config['train_params']['path'],sample_data=sample)
    num_epochs = config['train_params']['epochs']
    optimizer = Adam(model.parameters(), lr=config['train_params']['lr'])
    # scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=1, verbose=True)
    criterion = {
        'l1': torch.nn.L1Loss(),
        'l2': torch.nn.MSELoss()
    }.get(config['train_params']['crit'])

    # Create output directories
    if not os.path.exists(config['train_params']['task_name']):
        os.mkdir(config['train_params']['task_name'])
    if not os.path.exists(os.path.join(config['train_params']['task_name'],
                                       config['train_params']['output_train_dir'])):
        os.mkdir(os.path.join(config['train_params']['task_name'],
                              config['train_params']['output_train_dir']))

    # Load checkpoint if found
    # if os.path.exists(os.path.join(config['train_params']['task_name'],
        #                                                 config['train_params']['ckpt_name'])):
        # print('Loading checkpoint')
        # model.load_state_dict(torch.load(os.path.join(config['train_params']['task_name'],
        #                                               config['train_params']['ckpt_name']), map_location=device))
    best_loss = np.inf

    for epoch_idx in range(num_epochs):
        mean_loss = train_for_one_epoch(epoch_idx, model, data_loader, optimizer, criterion, config)
        # scheduler.step(mean_loss)
        # Simply update checkpoint if found better version
        if mean_loss < best_loss:
            print('Improved Loss to {:.4f} .... Saving Model'.format(mean_loss))
            torch.save(model.state_dict(), os.path.join(config['train_params']['task_name'],
                                                        config['train_params']['ckpt_name']))
            best_loss = mean_loss
        else:
            print('No Loss Improvement')

    #torch.save(indices_tensor, 'quantized_indices.pt')
    embedding_weights = model.quantizer.embedding.weight.detach().cpu()
    #torch.save(embedding_weights, 'learned_codebook.pt')
    return model
# def save_embedding_weights(model, file_path):
#     # Save the learned embedding weights
#     embedding_weights = model.quantizer.embedding.weight.detach().cpu()
#     torch.save(embedding_weights, file_path)

# def load_embedding_weights(file_path):
#     # Load the saved embedding weights
#     return torch.load(file_path)

# def get_quantized_output_from_indices(model, indices):
#     # Get quantized output from indices
#     quantized_output = model.quantizer.quantize_indices(indices)
#     return quantized_output


model= train(config_path='hyperparameters.yaml')
# kf = KFold(n_splits=5, shuffle=True, random_state=seed)
# for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
#     print(f"Fold {fold + 1}")

#     # Create data samplers
#     train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
#     val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

#     # Create data loaders
#     train_loader = load_data(config['train_params']['path'],sample_data=train_sampler)
#     val_loader = load_data(config['train_params']['path'],sample_data=val_sampler)

{'model_params': {'in_channels': 3, 'convbn_blocks': 4, 'conv_kernel_size': [3, 3, 3, 2], 'conv_kernel_strides': [2, 2, 1, 1], 'convbn_channels': [3, 6, 12, 24, 72], 'conv_activation_fn': 'relu', 'transpose_bn_blocks': 4, 'transposebn_channels': [72, 24, 12, 6, 3], 'transpose_kernel_size': [3, 3, 3, 2], 'transpose_kernel_strides': [2, 2, 1, 1], 'transpose_activation_fn': 'relu', 'latent_dim': 72, 'codebook_size': 10}, 'train_params': {'task_name': 'vqvae_latent_72_codebook_10_nnLayers_4', 'batch_size': 72, 'epochs': 10, 'lr': 0.005, 'crit': 'l2', 'reconstruction_loss_weight': 1, 'codebook_loss_weight': 1, 'commitment_loss_weight': 0.2, 'ckpt_name': 'best_vqvae_latent_72_codebook_10.pth', 'seed': 42, 'save_training_image': True, 'path': '/content/drive/MyDrive/Data/3dshapes.h5', 'output_train_dir': 'output'}}


Finished epoch: 1 | Recon Loss : 0.1149 | Codebook Loss : 0.3571 | Commitment Loss : 0.3571
Improved Loss to 0.5435 .... Saving Model


Finished epoch: 2 | Recon Loss : 0.0767 | Codebook Loss : 0.3223 | Commitment Loss : 0.3223
Improved Loss to 0.4634 .... Saving Model


Finished epoch: 3 | Recon Loss : 0.0710 | Codebook Loss : 0.2797 | Commitment Loss : 0.2797
Improved Loss to 0.4066 .... Saving Model


Finished epoch: 4 | Recon Loss : 0.0707 | Codebook Loss : 0.2868 | Commitment Loss : 0.2868
No Loss Improvement


...

Finished epoch: 6 | Recon Loss : 0.0651 | Codebook Loss : 0.2652 | Commitment Loss : 0.2652
Improved Loss to 0.3833 .... Saving Model


Finished epoch: 7 | Recon Loss : 0.0633 | Codebook Loss : 0.2540 | Commitment Loss : 0.2540
Improved Loss to 0.3681 .... Saving Model


Finished epoch: 8 | Recon Loss : 0.0623 | Codebook Loss : 0.2582 | Commitment Loss : 0.2582
No Loss Improvement


Finished epoch: 9 | Recon Loss : 0.0604 | Codebook Loss : 0.2495 | Commitment Loss : 0.2495
Improved Loss to 0.3597 .... Saving Model


Finished epoch: 10 | Recon Loss : 0.0599 | Codebook Loss : 0.2480 | Commitment Loss : 0.2480
Improved Loss to 0.3575 .... Saving Model


In [10]:
model.parameters()


<generator object Module.parameters at 0x7bf4cb8fd3f0>

In [14]:
# upload external file before import
from google.colab import files


In [15]:
torch.save(model.state_dict(), '3d400.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
state_dict = torch.load('3d400.pth')
print(state_dict.keys())

odict_keys(['encoder.encoder_layers.0.0.weight', 'encoder.encoder_layers.0.0.bias', 'encoder.encoder_layers.0.1.weight', 'encoder.encoder_layers.0.1.bias', 'encoder.encoder_layers.0.1.running_mean', 'encoder.encoder_layers.0.1.running_var', 'encoder.encoder_layers.0.1.num_batches_tracked', 'encoder.encoder_layers.1.0.weight', 'encoder.encoder_layers.1.0.bias', 'encoder.encoder_layers.1.1.weight', 'encoder.encoder_layers.1.1.bias', 'encoder.encoder_layers.1.1.running_mean', 'encoder.encoder_layers.1.1.running_var', 'encoder.encoder_layers.1.1.num_batches_tracked', 'encoder.encoder_layers.2.0.weight', 'encoder.encoder_layers.2.0.bias', 'encoder.encoder_layers.2.1.weight', 'encoder.encoder_layers.2.1.bias', 'encoder.encoder_layers.2.1.running_mean', 'encoder.encoder_layers.2.1.running_var', 'encoder.encoder_layers.2.1.num_batches_tracked', 'encoder.encoder_layers.3.0.weight', 'encoder.encoder_layers.3.0.bias', 'encoder.encoder_layers.3.1.weight', 'encoder.encoder_layers.3.1.bias', 'encode

In [19]:
config_path='hyperparameters.yaml'
with open(config_path, 'r') as file:
      config = yaml.safe_load(file)

model2 = get_model(config).to(device)

{'model_params': {'in_channels': 3, 'convbn_blocks': 4, 'conv_kernel_size': [3, 3, 3, 2], 'conv_kernel_strides': [2, 2, 1, 1], 'convbn_channels': [3, 6, 12, 24, 72], 'conv_activation_fn': 'relu', 'transpose_bn_blocks': 4, 'transposebn_channels': [72, 24, 12, 6, 3], 'transpose_kernel_size': [3, 3, 3, 2], 'transpose_kernel_strides': [2, 2, 1, 1], 'transpose_activation_fn': 'relu', 'latent_dim': 72, 'codebook_size': 10}, 'train_params': {'task_name': 'vqvae_latent_72_codebook_10_nnLayers_4', 'batch_size': 72, 'epochs': 10, 'lr': 0.005, 'crit': 'l2', 'reconstruction_loss_weight': 1, 'codebook_loss_weight': 1, 'commitment_loss_weight': 0.2, 'ckpt_name': 'best_vqvae_latent_72_codebook_10.pth', 'seed': 42, 'save_training_image': True, 'path': '/content/drive/MyDrive/Data/3dshapes.h5', 'output_train_dir': 'output'}}


In [20]:
model2.load_state_dict(state_dict)

<All keys matched successfully>

In [21]:
model2.quantizer.embedding.weight.shape

torch.Size([10, 72])

In [ ]:
# Load the tensor from the file
loaded_tensor = torch.load('learned_codebook.pt')

# Print the loaded tensor to verify
print(loaded_tensor.shape)

torch.Size([10, 72])
